In [7]:
import re
import PyPDF2
import os
import glob
import json
from tqdm import tqdm
import pandas as pd
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def extract_text_from_pdf(pdf_path):
    """Mengekstrak teks dari file PDF dengan cleaning watermark, disclaimer, dll."""
    try:
        logger.info(f"Reading PDF: {pdf_path}")
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text() or ""
                text += page_text + "\n"

            # Hapus watermark dan disclaimer tetap dari Mahkamah Agung
            text = re.sub(r'Mahkamah Agung Republik Indonesia\n', '', text)
            text = re.sub(r'Disclaimer\n', '', text)
            text = re.sub(r'Kepaniteraan Mahkamah Agung Republik Indonesia.*?kewaktu\.', '', text, flags=re.DOTALL)
            text = re.sub(r'Dalam hal Anda menemukan.*?ext\.318\)', '', text, flags=re.DOTALL)

            # Hapus baris kosong berlebih, nomor halaman, dsb
            text = re.sub(r'\n\s*\n', '\n', text)
            text = re.sub(r'Halaman\s+\d+\s+dari\s+\d+', '', text, flags=re.I)
            text = re.sub(r'Halaman \d+', '', text, flags=re.I)

            # Normalisasi teks
            text = re.sub(r'\s+', ' ', text).strip()

            return text
    except Exception as e:
        logger.error(f"Error reading {pdf_path}: {str(e)}")
        return None

def detect_case_type(text):
    """Deteksi jenis putusan: pidana atau perdata"""
    if re.search(r'pdt\.|permohonan|pemohon|gugatan|penggugat|tergugat', text, re.IGNORECASE):
        return 'perdata'
    elif re.search(r'pid\.|terdakwa|jaksa|penuntut umum|tindak pidana', text, re.IGNORECASE):
        return 'pidana'
    return 'unknown'

def extract_pidana_info(text):
    """Extractor untuk putusan pidana penganiayaan"""
    info = {
        'jenis_putusan': 'pidana',
        'nomor_putusan': '',
        'nama_terdakwa': '',
        'tempat_tanggal_lahir': '',
        'umur': '',
        'jenis_kelamin': '',
        'kebangsaan': '',
        'agama': '',
        'alamat': '',
        'pekerjaan': '',
        'tindak_pidana': '',
        'pasal_didakwa': '',
        'kronologi': [],
        'keterangan_saksi': [],
        'visum_et_repertum': '',
        'barang_bukti': [],
        'tuntutan_jaksa': '',
        'pembelaan_terdakwa': '',
        'pertimbangan_hukum': '',
        'putusan': '',
        'lama_hukuman': '',
        'jenis_hukuman': '',
        'masa_penahanan': '',
        'keringanan': [],
        'memberatkan': [],
        'biaya_perkara': '',
        'tanggal_putusan': '',
        'tanggal_kejadian': '',
        'lokasi_kejadian': ''
    }

    # Ekstrak nomor putusan
    nomor_match = re.search(r'Nomor[:\s]*([\d/]+/Pid\.?[A-Z]?/\d+/PN[^,\n]*)', text, re.IGNORECASE)
    if nomor_match:
        info['nomor_putusan'] = nomor_match.group(1).strip()

    # Ekstrak data terdakwa
    nama_match = re.search(r'Nama (?:Lengkap|Terdakwa)[:\s]*([^\n;]+)', text, re.IGNORECASE)
    if nama_match:
        info['nama_terdakwa'] = nama_match.group(1).strip()

    ttl_match = re.search(r'Tempat[/\s]*(?:Tgl\.?|tgı) Lahir[:\s]*([^\n;]+)', text, re.IGNORECASE)
    if ttl_match:
        info['tempat_tanggal_lahir'] = ttl_match.group(1).strip()

    umur_match = re.search(r'Umur[:\s]*(\d+)\s*tahun', text, re.IGNORECASE)
    if umur_match:
        info['umur'] = umur_match.group(1).strip()

    jk_match = re.search(r'Jenis Kelamin[:\s]*([^\n;]+)', text, re.IGNORECASE)
    if jk_match:
        info['jenis_kelamin'] = jk_match.group(1).strip()

    bangsa_match = re.search(r'Kebangsaan[:\s]*([^\n;]+)', text, re.IGNORECASE)
    if bangsa_match:
        info['kebangsaan'] = bangsa_match.group(1).strip()

    agama_match = re.search(r'Agama[:\s]*([^\n;]+)', text, re.IGNORECASE)
    if agama_match:
        info['agama'] = agama_match.group(1).strip()

    alamat_match = re.search(r'Tempat Tinggal[:\s]*([^\n;]+(?:\n[^\n;]+)*)', text, re.IGNORECASE)
    if alamat_match:
        info['alamat'] = ' '.join(alamat_match.group(1).split())

    pekerjaan_match = re.search(r'Pekerjaan[:\s]*([^\n;]+)', text, re.IGNORECASE)
    if pekerjaan_match:
        info['pekerjaan'] = pekerjaan_match.group(1).strip()

    # Ekstrak jenis tindak pidana
    pidana_match = re.search(r'melakukan Tindak Pidana\s*([^;\.]+)', text, re.IGNORECASE)
    if pidana_match:
        info['tindak_pidana'] = pidana_match.group(1).strip()

    # Ekstrak pasal yang didakwa
    pasal_match = re.search(r'(Pasal\s+\d+\s*(?:ayat\s*\(\d+\))?\s*KUHP(?:idana)?)', text, re.IGNORECASE)
    if pasal_match:
        info['pasal_didakwa'] = pasal_match.group(1).strip()

    # Ekstrak kronologi
    kronologi_match = re.search(r'telah diperoleh fakta-fakta sebagai berikut:(.*?)Menimbang, bahwa dengan pertimbangan', text, re.DOTALL | re.IGNORECASE)
    if kronologi_match:
        kronologi_text = kronologi_match.group(1)
        facts = re.findall(r'Bahwa[^;]+;', kronologi_text)
        info['kronologi'] = [fact.strip() for fact in facts]

    # Ekstrak keterangan saksi
    saksi_pattern = r'Saksi[^:]*?:\s*([^;]+?(?=Saksi|Menimbang|$))'
    saksi_matches = re.findall(saksi_pattern, text, re.DOTALL | re.IGNORECASE)
    info['keterangan_saksi'] = [s.strip() for s in saksi_matches if s.strip()]

    # Ekstrak visum et repertum
    visum_match = re.search(r'VISUM ET REPERTUM[^:]*?:\s*([^;]+?(?=Menimbang|$))', text, re.DOTALL | re.IGNORECASE)
    if visum_match:
        info['visum_et_repertum'] = visum_match.group(1).strip()

    # Ekstrak barang bukti
    bukti_match = re.search(r'Barang Bukti[^:]*?:\s*([^;]+?(?=Menimbang|$))', text, re.DOTALL | re.IGNORECASE)
    if bukti_match:
        bukti_text = bukti_match.group(1)
        bukti_items = re.findall(r'[a-zA-Z][^,;]*', bukti_text)
        info['barang_bukti'] = [b.strip() for b in bukti_items if b.strip()]

    # Ekstrak tuntutan jaksa
    tuntutan_match = re.search(r'Tuntutan[^:]*?:\s*([^;]+?(?=Pembelaan|Menimbang|$))', text, re.DOTALL | re.IGNORECASE)
    if tuntutan_match:
        info['tuntutan_jaksa'] = tuntutan_match.group(1).strip()

    # Ekstrak pembelaan terdakwa
    pembelaan_match = re.search(r'Pembelaan[^:]*?:\s*([^;]+?(?=Menimbang|$))', text, re.DOTALL | re.IGNORECASE)
    if pembelaan_match:
        info['pembelaan_terdakwa'] = pembelaan_match.group(1).strip()

    # Ekstrak pertimbangan hukum
    pertimbangan_match = re.search(r'PERTIMBANGAN HUKUM(.*?)MENGADILI', text, re.DOTALL | re.IGNORECASE)
    if pertimbangan_match:
        info['pertimbangan_hukum'] = pertimbangan_match.group(1).strip()

    # Ekstrak putusan
    putusan_match = re.search(r'MENGADILI[:\s]*(.*?)(?=Demikianlah|$)', text, re.DOTALL | re.IGNORECASE)
    if putusan_match:
        info['putusan'] = putusan_match.group(1).strip()

    # Ekstrak lama hukuman
    hukuman_match = re.search(r'pidana\s+penjara\s+selama\s+(\d+\s+\w+)', text, re.IGNORECASE)
    if hukuman_match:
        info['lama_hukuman'] = hukuman_match.group(1).strip()

    # Ekstrak jenis hukuman
    jenis_hukuman_match = re.search(r'jenis\s+pidana[^:]*?:\s*([^;]+)', text, re.IGNORECASE)
    if jenis_hukuman_match:
        info['jenis_hukuman'] = jenis_hukuman_match.group(1).strip()

    # Ekstrak masa penahanan
    tahanan_match = re.search(r'masa\s+penahanan[^:]*?:\s*([^;]+)', text, re.IGNORECASE)
    if tahanan_match:
        info['masa_penahanan'] = tahanan_match.group(1).strip()

    # Ekstrak keadaan meringankan
    keringanan_matches = re.findall(r'keadaan\s+yang\s+meringankan[^:]*?[:\-]\s*([^;\.]+)', text, re.IGNORECASE)
    info['keringanan'] = [k.strip() for k in keringanan_matches if k.strip()]

    # Ekstrak keadaan memberatkan
    memberatkan_matches = re.findall(r'keadaan\s+yang\s+memberatkan[^:]*?[:\-]\s*([^;\.]+)', text, re.IGNORECASE)
    info['memberatkan'] = [m.strip() for m in memberatkan_matches if m.strip()]

    # Ekstrak biaya perkara
    biaya_match = re.search(r'biaya\s+perkara.*?Rp\s*([\d.,]+)', text, re.IGNORECASE)
    if biaya_match:
        info['biaya_perkara'] = 'Rp ' + biaya_match.group(1).strip()

    # Ekstrak tanggal putusan
    tgl_putusan_match = re.search(r'diputuskan.*?tanggal\s+(\d+\s+\w+\s+\d{4})', text, re.IGNORECASE)
    if tgl_putusan_match:
        info['tanggal_putusan'] = tgl_putusan_match.group(1).strip()

    # Ekstrak tanggal kejadian
    tgl_kejadian_match = re.search(r'pada\s+hari.*?tanggal\s+(\d+\s+\w+\s+\d{4})', text, re.IGNORECASE)
    if tgl_kejadian_match:
        info['tanggal_kejadian'] = tgl_kejadian_match.group(1).strip()

    # Ekstrak lokasi kejadian
    lokasi_match = re.search(r'di\s+([^,.;]+(?:\s+[^,.;]+)*)', text)
    if lokasi_match and 'jalan' in lokasi_match.group(1).lower():
        info['lokasi_kejadian'] = lokasi_match.group(1).strip()

    return info

def extract_case_info(text):
    """Deteksi jenis putusan dan ekstrak informasi yang sesuai"""
    case_type = detect_case_type(text)
    logger.info(f"Detected case type: {case_type}")

    if case_type == 'pidana':
        return extract_pidana_info(text)
    else:
        logger.warning("Non-pidana case detected, skipping detailed extraction")
        return {'jenis_putusan': case_type, 'error': 'Bukan putusan pidana'}

def process_pdfs(input_folder, output_folder):
    """Memproses semua PDF dalam folder input dan menyimpan hasilnya di folder output"""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Dapatkan semua file PDF dalam folder input
    pdf_files = glob.glob(os.path.join(input_folder, "*.pdf"))

    # Inisialisasi list untuk menyimpan semua data
    all_cases_data = []

    # Process each PDF file
    for i, pdf_file in enumerate(tqdm(pdf_files, desc="Processing PDFs", unit="file")):
        logger.info(f"Processing file {i+1}: {pdf_file}")
        # Ekstrak teks dari PDF
        text = extract_text_from_pdf(pdf_file)

        if text:
            # Ekstrak informasi dari teks
            case_info = extract_case_info(text)

            # Tambahkan nama file asli ke informasi
            case_info['file_source'] = os.path.basename(pdf_file)

            # Simpan ke list
            all_cases_data.append(case_info)

            # Buat nama file output
            output_filename = f"case{i+1:03d}.json"
            output_path = os.path.join(output_folder, output_filename)

            # Simpan sebagai JSON
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(case_info, f, ensure_ascii=False, indent=2)
        else:
            logger.warning(f"No text extracted from {pdf_file}")

    # Simpan semua data dalam satu file CSV untuk analisis
    csv_output_path = os.path.join(output_folder, "all_cases.csv")
    df = pd.DataFrame(all_cases_data)
    df.to_csv(csv_output_path, index=False, encoding='utf-8')

    return all_cases_data

if __name__ == "__main__":
    input_folder = "data/raw/PDF"  # Ganti dengan path folder PDF Anda
    output_folder = "data/raw/TXT"   # Ganti dengan path folder output Anda

    cases_data = process_pdfs(input_folder, output_folder)
    print(f"Ekstraksi selesai! {len(cases_data)} file diproses.")

Processing PDFs:   0%|          | 0/1003 [00:00<?, ?file/s]2025-09-17 12:43:28,489 - INFO - Processing file 1: data/raw/PDF\putusan_1004_pdt.g_2025_pa.pmk_20250819132455.pdf
2025-09-17 12:43:28,490 - INFO - Reading PDF: data/raw/PDF\putusan_1004_pdt.g_2025_pa.pmk_20250819132455.pdf
2025-09-17 12:43:28,791 - INFO - Detected case type: perdata
2025-09-17 12:43:28,793 - WARNING - Non-pidana case detected, skipping detailed extraction
Processing PDFs:   0%|          | 1/1003 [00:00<05:05,  3.28file/s]2025-09-17 12:43:28,794 - INFO - Processing file 2: data/raw/PDF\putusan_1008_pid.b_2024_pn_plg_20250820090800.pdf
2025-09-17 12:43:28,795 - INFO - Reading PDF: data/raw/PDF\putusan_1008_pid.b_2024_pn_plg_20250820090800.pdf
2025-09-17 12:43:29,129 - INFO - Detected case type: perdata
2025-09-17 12:43:29,129 - WARNING - Non-pidana case detected, skipping detailed extraction
Processing PDFs:   0%|          | 2/1003 [00:00<05:23,  3.09file/s]2025-09-17 12:43:29,131 - INFO - Processing file 3: dat

Ekstraksi selesai! 1003 file diproses.
